# Trabalho Final de Mineração de Dados

## Importando bibliotecas

In [1]:
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.dummy import DummyClassifier
from imblearn.under_sampling import RandomUnderSampler

import pandas as pd
import numpy as np
import seaborn as sns
import nltk
import string
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\paulo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\paulo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## 1 - Fase de Preparação dos Dados
---

Carregando o arquivo original de classes.

In [2]:
classes_original = pd.read_excel("dados/SistemaDeClassificacao.xlsx", "Select viw_classificacao_arvore")
classes_original.head()

,,COD_CLASSE,DES_NOME_PREFERIDO,COD_CLASSE_PAI,NUM_NIVEL,CYCLE,TREE,PATH,QTD_FILHOS,DES_NIVEL1,DES_NIVEL2,DES_NIVEL3,DES_NIVEL4
0,1,33254494,Classificação Temática Unificada,NaN,1,0,Classificação Temática Unificada,Classificação Temática Unificada,202,NaN,NaN,NaN,NaN
1,2,33809814,Temas Exclusivos de Pronunciamentos,33254494.0,2,0,-- Temas Exclusivos de Pronunciamentos,Classificação Temática Unificada / Temas Exclu...,22,Temas Exclusivos de Pronunciamentos,NaN,NaN,NaN
2,3,33809634,Meio Ambiente,33254494.0,2,0,-- Meio Ambiente,Classificação Temática Unificada / Meio Ambiente,11,Meio Ambiente,NaN,NaN,NaN
3,4,33809514,"Soberania, Defesa Nacional e Ordem Pública",33254494.0,2,0,"-- Soberania, Defesa Nacional e Ordem Pública","Classificação Temática Unificada / Soberania, ...",7,"Soberania, Defesa Nacional e Ordem Pública",NaN,NaN,NaN
4,5,33808912,Política Social,33254494.0,2,0,-- Política Social,Classificação Temática Unificada / Política So...,37,Política Social,NaN,NaN,NaN


Filtra o arquivo de classes original para exibir apenas aquelas que serão raiz na classificação da ementa.
Essa classes possuem nível 2 na tabela original.

In [3]:
classes_raiz = classes_original.query("NUM_NIVEL == 2").filter(["COD_CLASSE", "DES_NOME_PREFERIDO"])
classes_raiz.rename(columns={"DES_NOME_PREFERIDO": "DES_CLASSE"}, inplace=True)
classes_raiz

,COD_CLASSE,DES_CLASSE
1,33809814,Temas Exclusivos de Pronunciamentos
2,33809634,Meio Ambiente
3,33809514,"Soberania, Defesa Nacional e Ordem Pública"
4,33808912,Política Social
5,33805362,Jurídico
6,33805317,Honorífico
7,33805137,Infraestrutura
8,33769167,Economia e Desenvolvimento
9,33768972,Organização do Estado
10,33685789,Administração Pública


Carrega o dataset contendo as normas que já foram classificadas. Com os dados carregados, cria uma coluna derivada (DES_CLASSE_RAIZ) a partir da árvore de classes (DES_CLASSE_HIERARQUIA), da qual a informação de classe raiz é extraída.

In [4]:
normas_classificadas_original = pd.read_excel("dados/ClassificacaoDeLeisOrdinarias-LeisComplementares-e-DecretosNumerados-Desde1900.xlsx", "Select mvw_u03_prc_doc_tema")
normas_classificadas_original["DES_CLASSE_RAIZ"] = normas_classificadas_original["DES_CLASSE_HIERARQUIA"].apply(lambda hierarquia : hierarquia.split(" / ")[1])
normas_classificadas_original.head()

,,COD_PRC_DOC_TEMA,COD_PROCESSO_DOCUMENTO,COD_CLASSE,DES_CLASSE,DES_CLASSE_HIERARQUIA,DES_CLASSE_RAIZ
0,1,36155183,386343,33805827,Crédito Suplementar,Classificação Temática Unificada / Orçamento P...,Orçamento Público
1,2,36192020,386579,33805287,Rádio e TV,Classificação Temática Unificada / Infraestrut...,Infraestrutura
2,3,36155185,387419,33805827,Crédito Suplementar,Classificação Temática Unificada / Orçamento P...,Orçamento Público
3,4,36192056,387832,33805287,Rádio e TV,Classificação Temática Unificada / Infraestrut...,Infraestrutura
4,5,36155187,388197,33805827,Crédito Suplementar,Classificação Temática Unificada / Orçamento P...,Orçamento Público


Carrega os dados de todas as normas (classificadas e não classificadas) e aplica as transformações iniciais.

In [5]:
normas_original = pd.read_excel("dados/LeisOrdinarias-LeisComplementare-e-DecretosNumeradosComClassificacaoDesde1900.xlsx", "Select mvw_s01_documento")
normas_original.rename(columns={"DBMS_LOB.SUBSTR(S01.TXT_EMENTA": "TXT_EMENTA"}, inplace=True)
normas_original.drop(columns="   ", inplace=True)
normas_original.head()

,COD_DOCUMENTO,DES_NOME_PREFERIDO,DES_NOMES_ALTERNATIVOS,TXT_EMENTA
0,35345364,Lei nº 14.263 de 22/12/2021,LEI-14263-2021-12-22,Abre ao Orçamento da Seguridade Social da Uniã...
1,26247104,Lei nº 13.486 de 03/10/2017,LEI-13486-2017-10-03,"Altera o art. 8º da Lei nº 8.078, de 11 de set..."
2,27445746,Lei nº 13.701 de 06/08/2018,LEI-13701-2018-08-06,Cria o cargo de natureza especial de Intervent...
3,36348502,Lei nº 14.447 de 09/09/2022,LEI-14447-2022-09-09,Altera os limites da Floresta Nacional de Bras...
4,32103727,Lei nº 13.988 de 14/04/2020,LEI-13988-2020-04-14,Dispõe sobre a transação nas hipóteses que esp...


In [6]:
normas_original.shape[0]

26959

Com os datasets necessários já carregados, cria-se um novo dataset a partir do dataset contendo todas as leis, incluindo-se a informação das classes contidadas nas normas do dataset de normas classificadas.

In [7]:
normas = normas_original.merge(normas_classificadas_original.filter(["COD_PROCESSO_DOCUMENTO", "DES_CLASSE_RAIZ"]), left_on="COD_DOCUMENTO", right_on="COD_PROCESSO_DOCUMENTO", how="left")
normas = normas.merge(classes_raiz, left_on="DES_CLASSE_RAIZ", right_on="DES_CLASSE", how="left")
normas.drop(columns=["COD_PROCESSO_DOCUMENTO", "DES_CLASSE_RAIZ"], inplace=True)
normas.head()

,COD_DOCUMENTO,DES_NOME_PREFERIDO,DES_NOMES_ALTERNATIVOS,TXT_EMENTA,COD_CLASSE,DES_CLASSE
0,35345364,Lei nº 14.263 de 22/12/2021,LEI-14263-2021-12-22,Abre ao Orçamento da Seguridade Social da Uniã...,33260515.0,Orçamento Público
1,26247104,Lei nº 13.486 de 03/10/2017,LEI-13486-2017-10-03,"Altera o art. 8º da Lei nº 8.078, de 11 de set...",33808912.0,Política Social
2,27445746,Lei nº 13.701 de 06/08/2018,LEI-13701-2018-08-06,Cria o cargo de natureza especial de Intervent...,33768972.0,Organização do Estado
3,36348502,Lei nº 14.447 de 09/09/2022,LEI-14447-2022-09-09,Altera os limites da Floresta Nacional de Bras...,33685789.0,Administração Pública
4,36348502,Lei nº 14.447 de 09/09/2022,LEI-14447-2022-09-09,Altera os limites da Floresta Nacional de Bras...,33809634.0,Meio Ambiente


In [8]:
normas.shape[0]

27743

Limpa os registros duplicados de normas que possuem mais de uma classe folha, mas que tenham mesma classe raiz e exibe as leis que possuem mais de uma classe folha, mas que tenham classes raiz diferentes.

In [9]:
normas.drop_duplicates(inplace=True)
temp = normas[["COD_DOCUMENTO", "COD_CLASSE", "DES_CLASSE"]].groupby("COD_DOCUMENTO")
temp.filter(lambda x: len(x) > 1)

,COD_DOCUMENTO,COD_CLASSE,DES_CLASSE
3,36348502,33685789.0,Administração Pública
4,36348502,33809634.0,Meio Ambiente
5,36348502,33769167.0,Economia e Desenvolvimento
20,36062349,33808912.0,Política Social
21,36062349,33769167.0,Economia e Desenvolvimento
...,...,...,...
27349,36032872,33808912.0,Política Social
27350,35556312,33685789.0,Administração Pública
27351,35556312,33808912.0,Política Social
27359,35396946,33809514.0,"Soberania, Defesa Nacional e Ordem Pública"


Tamanho da base tratada de normas.

In [10]:
normas.shape[0]

27294

Total de classificações feitas em normas.

In [11]:
normas_classificadas = normas.query("not COD_CLASSE.isnull()")
quantidade_classificacoes = normas_classificadas.shape[0]
quantidade_classificacoes

17438

Número distinto de normas classificadas.

In [12]:
len(normas_classificadas["COD_DOCUMENTO"].unique())

17103

Quantidade de normas a classificar.

In [13]:
normas_nao_classificadas = normas.query("COD_CLASSE.isnull()")
normas_nao_classificadas.shape[0]

9856

Verificação da distribuição das classes (em %).

In [14]:
frequencia = normas_classificadas["DES_CLASSE"].value_counts()
percentual = normas_classificadas["DES_CLASSE"].value_counts(normalize=True) * 100
distribuicao_classes = pd.DataFrame({'Frequência': frequencia, 'Percentual (%)': percentual})
distribuicao_classes.sort_values(by="Frequência", ascending=False)

,Frequência,Percentual (%)
Orçamento Público,11192,64.181672
Infraestrutura,2508,14.382383
Política Social,877,5.029246
Administração Pública,783,4.490194
Honorífico,671,3.847918
Economia e Desenvolvimento,644,3.693084
Jurídico,381,2.184884
"Soberania, Defesa Nacional e Ordem Pública",166,0.951944
Organização do Estado,130,0.745498
Meio Ambiente,86,0.493176


## 2 - Estruturação do texto
***

Definição das funções para a tarefa de criação das matrizes de documentos (texto da ementa)

In [15]:
# Divide um texto em tokens utilizando a biblioteca de NLP nltk
def tokenizador(texto):
    texto = texto.lower()
    lista_alfanumerica = []

    for token_valido in nltk.word_tokenize(texto, language="portuguese"):
        # Se for caracter de pontuação ou for stopword
        if token_valido in string.punctuation:
            continue
        # elif token_valido in nltk.corpus.stopwords.words('portuguese'):
        #     continue

        lista_alfanumerica.append(token_valido)

    return lista_alfanumerica

In [16]:
# Combinação de vetores por soma
def combinacao_vetores(palavras_numeros, skipgram):
    vetor_resultante = np.zeros(300)

    for pn in palavras_numeros:
        try:
            vetor_resultante += skipgram.get_vector(pn)
        except KeyError:
            if pn.isnumeric():
                pn = "0" * len(pn) # "0, 00, 000, 0000, etc. dependendo do tamanho de caracteres do número
                vetor_resultante += skipgram.get_vector(pn)
            else:
                vetor_resultante += skipgram.get_vector("unknown")

    return vetor_resultante

In [17]:
def matriz_word_embeddings(textos, skipgram):
    x = len(textos)
    y = 300 # número de dimensões do skipgram (skip300)
    matriz = np.zeros((x, y))

    for i in range(x):
        palavras_numeros = tokenizador(textos.iloc[i])
        matriz[i] = combinacao_vetores(palavras_numeros, skipgram)

    return matriz

Carrega o word embeddings (Word2Vec - Skipgram) pré-treinado pelo NILC (Núcleo Interinstitucional de Linguística Computacional).

In [18]:
# Carrega o word embeddings skip-gram
skipgram = KeyedVectors.load_word2vec_format("dados/skip_s300.txt")

Criação da matriz de documentos do corpus (conjunto de ementas), aqui representada por matrizes de word embeddings (cada ementa é vetorizada assim como o word embeddings pré-treinado). Também é feita a separação das classes (labels).

In [19]:
X = matriz_word_embeddings(normas_classificadas["TXT_EMENTA"], skipgram)
y = normas_classificadas["DES_CLASSE"]

## 3 - Treinamento e predição do modelo de Machine Learning para classificação

Separação inicial dos dados. Para a avaliação inicial dos modelos de ML, a proporção será de 70% para treino e 30% para testes.

In [20]:
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

### 3.1 - Comparação de modelos

Modelos de classificação a serem comparados.

In [21]:
modelos = {"Dummy": DummyClassifier(random_state=42),
           "BernoulliNB": BernoulliNB(binarize=np.median(X_treino)),
           "KNN 1": KNeighborsClassifier(n_neighbors=1),
           "KNN 3": KNeighborsClassifier(n_neighbors=3),
           "KNN 5": KNeighborsClassifier(),
           "KNN 11": KNeighborsClassifier(n_neighbors=11),
           "DecisionTree": DecisionTreeClassifier(class_weight="balanced", random_state=42),
           "SVC": SVC(class_weight="balanced", random_state=42),
           "RandomForest": RandomForestClassifier(class_weight="balanced", random_state=42),
           "LogisticRegression": LogisticRegression(max_iter=6000, class_weight="balanced", random_state=42)}

Definindo a função para execução dos modelos.

In [22]:
def executar_modelo(modelo, X_treino, y_treino, X_teste, y_teste):
    modelo.fit(X_treino, y_treino)
    y_previsto = modelo.predict(X_teste)
    return accuracy_score(y_teste, y_previsto)

Executando os modelos de Machine Learning para avaliação.

In [23]:
scores = []
for nome, modelo in modelos.items():
    score = executar_modelo(modelo, X_treino, y_treino, X_teste, y_teste)
    scores.append(score)
    print(f"{nome}: {score}")

Dummy: 0.6418195718654435
BernoulliNB: 0.8119266055045872
KNN 1: 0.8814984709480123
KNN 3: 0.8904816513761468
KNN 5: 0.8918195718654435
KNN 11: 0.8900993883792049
DecisionTree: 0.8232033639143731
SVC: 0.8906727828746177
RandomForest: 0.8811162079510704
LogisticRegression: 0.8950688073394495


Treinamento e predição do classificador por Regressão Logística.

In [24]:
lr = LogisticRegression(max_iter=6000, class_weight="balanced", random_state=42)
lr.fit(X_treino, y_treino)
y_previsto = lr.predict(X_teste)

### 3.2 - Avaliando o modelo de maior acurácia

Exibe os relatórios de desempenho do modelo de maior acurácia (LogisticRegression).

In [25]:
acuracia = accuracy_score(y_teste, y_previsto)
print("Acurácia LogisticRegression:",  round(acuracia, 2))

cr = classification_report(y_teste, y_previsto)
print(cr)

Acurácia LogisticRegression: 0.9
                                            precision    recall  f1-score   support

                     Administração Pública       0.61      0.54      0.57       235
                Economia e Desenvolvimento       0.50      0.59      0.54       193
                                Honorífico       0.92      0.88      0.90       201
                            Infraestrutura       0.96      0.96      0.96       753
                                  Jurídico       0.45      0.49      0.47       114
                             Meio Ambiente       0.40      0.38      0.39        26
                     Organização do Estado       0.17      0.31      0.22        39
                         Orçamento Público       1.00      0.98      0.99      3358
                           Política Social       0.61      0.60      0.60       263
Soberania, Defesa Nacional e Ordem Pública       0.26      0.32      0.29        50

                                  accurac

### 3.3 - Criando _baseline_ do dataset com redução das classes majoritárias

Para tentar diminuir o desbalanceamento dos dados, o número de amostras das duas classes majoritárias (Orçamento Público e Infraestrutura) será diminuído para 1000 cada.

In [26]:
rus = RandomUnderSampler(sampling_strategy={"Orçamento Público": 1000, "Infraestrutura": 1000}, random_state=42)
X_res, y_res = rus.fit_resample(X, y)

frequencia_res = y_res.value_counts()
percentual_res = y_res.value_counts(normalize=True) * 100
distribuicao_classes_res = pd.DataFrame({'Frequência': frequencia_res, 'Percentual (%)': percentual_res})
distribuicao_classes_res.sort_values(by="Frequência", ascending=False)

,Frequência,Percentual (%)
Infraestrutura,1000,17.427675
Orçamento Público,1000,17.427675
Política Social,877,15.284071
Administração Pública,783,13.645870
Honorífico,671,11.693970
Economia e Desenvolvimento,644,11.223423
Jurídico,381,6.639944
"Soberania, Defesa Nacional e Ordem Pública",166,2.892994
Organização do Estado,130,2.265598
Meio Ambiente,86,1.498780


Dividindo o dataset na mesma proporção treino/teste anterior (70%/30%).

In [27]:
X_treino_res, X_teste_res, y_treino_res, y_teste_res = train_test_split(X_res, y_res, test_size=0.3, stratify=y_res, random_state=42)

Comparando modelos com o novo dataset reduzido.

In [28]:
scores = []
for nome, modelo in modelos.items():
    score = executar_modelo(modelo, X_treino_res, y_treino_res, X_teste_res, y_teste_res)
    scores.append(score)
    print(f"{nome}: {score}")

Dummy: 0.17421602787456447
BernoulliNB: 0.6347270615563299
KNN 1: 0.6817653890824622
KNN 3: 0.705574912891986
KNN 5: 0.7154471544715447
KNN 11: 0.7038327526132404
DecisionTree: 0.554006968641115
SVC: 0.7212543554006968
RandomForest: 0.7003484320557491
LogisticRegression: 0.7131242740998839


Treinando e avaliando o novo modelo (LogisticRegression) com dataset reduzido.

In [29]:
lr_res = LogisticRegression(max_iter=6000, class_weight="balanced", random_state=42)
lr_res.fit(X_treino_res, y_treino_res)
y_previsto_res = lr_res.predict(X_teste_res)

In [30]:
acuracia_res = accuracy_score(y_teste_res, y_previsto_res)
print("Acurácia LogisticRegression:", round(acuracia_res, 2))

cr_res = classification_report(y_teste_res, y_previsto_res)
print(cr_res)

Acurácia LogisticRegression: 0.71
                                            precision    recall  f1-score   support

                     Administração Pública       0.59      0.53      0.56       235
                Economia e Desenvolvimento       0.57      0.53      0.55       193
                                Honorífico       0.89      0.93      0.91       202
                            Infraestrutura       0.94      0.93      0.93       300
                                  Jurídico       0.48      0.56      0.52       114
                             Meio Ambiente       0.36      0.35      0.35        26
                     Organização do Estado       0.24      0.49      0.32        39
                         Orçamento Público       0.98      0.97      0.98       300
                           Política Social       0.57      0.52      0.55       263
Soberania, Defesa Nacional e Ordem Pública       0.28      0.30      0.29        50

                                  accura